In [12]:
import os
import asyncio
import httpx
from datetime import datetime
from typing import List, Optional, Union
from pydantic import BaseModel


# =========================
# Data model (Search only)
# =========================

class SearchResult(BaseModel):
    url: str
    title: Optional[str] = None
    snippet: Optional[str] = None
    engine: Optional[str] = None
    score: Optional[float] = None
    extracted_at: str


class SearchError(BaseModel):
    query: str
    error: str
    extracted_at: str
    success: bool = False


# =========================
# SearxNG Search Step 1
# =========================

class SearxNGSearch:
    def __init__(self, base_url: Optional[str] = None):
        self.base_url = base_url or os.getenv("SEARXNG_BASE_URL", "http://localhost:8888/search")
        if not self.base_url:
            raise ValueError("SEARXNG_BASE_URL is not set")

    async def _fetch_page(
        self,
        client: httpx.AsyncClient,
        query: str,
        page: int,
        timeout: int,
    ) -> Union[List[dict], str]:
        try:
            params = {
                "q": query,
                "format": "json",
                "pageno": page,
            }
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
                "Accept": "application/json",
            }
            r = await client.get(self.base_url, params=params, timeout=timeout, headers=headers)
            r.raise_for_status()
            return r.json().get("results", [])
        except Exception as e:
            return f"Page {page} failed: {e}"

    async def _search_async(
        self,
        query: str,
        max_pages: int,
        timeout: int,
        max_results: Optional[int],
        output_for_llm: bool,
    ) -> Union[List[SearchResult], str, SearchError]:

        collected: List[SearchResult] = []

        async with httpx.AsyncClient() as client:
            tasks = [
                self._fetch_page(client, query, page, timeout)
                for page in range(1, max_pages + 1)
            ]
            pages = await asyncio.gather(*tasks)

        for page in pages:
            if isinstance(page, str):
                return SearchError(
                    query=query,
                    error=page,
                    extracted_at=datetime.now().isoformat(),
                )

            for item in page:
                collected.append(
                    SearchResult(
                        url=item.get("url"),
                        title=item.get("title"),
                        snippet=item.get("content"),  # SearxNG snippet
                        engine=item.get("engine"),
                        score=item.get("score"),
                        extracted_at=datetime.now().isoformat(),
                    )
                )

        if max_results:
            collected.sort(
                key=lambda x: x.score if x.score is not None else 0,
                reverse=True,
            )
            collected = collected[:max_results]

        if output_for_llm:
            return "\n".join(r.model_dump_json() for r in collected)

        return collected

    async def asearch(
        self,
        query: str,
        *,
        max_pages: int = 3,
        timeout: int = 10,
        max_results: Optional[int] = None,
        output_for_llm: bool = False,
    ):
        return await self._search_async(
            query=query,
            max_pages=max_pages,
            timeout=timeout,
            max_results=max_results,
            output_for_llm=output_for_llm,
        )

    def search(
        self,
        query: str,
        *,
        max_pages: int = 3,
        timeout: int = 10,
        max_results: Optional[int] = None,
        output_for_llm: bool = False,
    ) -> Union[List[SearchResult], str, SearchError]:

        return asyncio.run(
            self._search_async(
                query=query,
                max_pages=max_pages,
                timeout=timeout,
                max_results=max_results,
                output_for_llm=output_for_llm,
            )
        )


# =========================
# Example usage (Step 1)
# =========================

if __name__ == "__main__":
    searcher = SearxNGSearch()

    results = await searcher.asearch(
    query="鹿島建設株主優待の有無・内容",
    max_pages=3,
    max_results=5,
    output_for_llm=False,   # 👈
    )

    content = []
    for result in results:
        content.append(result.snippet)
        print(result)
    text = "\n\n".join(f"- {c.strip()}" for c in content)
    print(text)



url='https://www.kajima.co.jp/ir/faq/?mode=pc' title='よくあるご質問（FAQ） | 鹿島建設株式会社' snippet='現在、株主優待制度は実施しておりません。 ページのトップへ戻る. Question 株主還元方針、配当金の推移について知りたい。 Answer. 配当\xa0...' engine='startpage' score=1.0 extracted_at='2026-01-08T15:08:16.923333'
url='https://www.jreast.co.jp/company/deal/border/' title='土地境界立会いについて｜法人お取引に関して - JR東日本' snippet='事前打ち合わせでは、当社グループ会社等が申請内容の確認を行います。 【当社グループ会社】 ○首都圏本部・横浜支社・八王子支社・大宮支社・高崎支社・水戸支社・千葉\xa0...' engine='startpage' score=1.0 extracted_at='2026-01-08T15:08:16.923577'
url='https://doda.jp/DodaFront/View/JobSearchList/j_oc__020314S/-op__1/-preBtn__3/' title='【知的財産（知財）・特許】新着の転職・求人・中途採用情報 doda ...' snippet='知的財産（知財）・特許、新着の転職・求人情報ページ：公開求人数69件。「完全週休2日制」「職種・業種未経験歓迎」など、条件にあった検索ができます。' engine='startpage' score=1.0 extracted_at='2026-01-08T15:08:16.923639'
url='https://www.kajima.co.jp/ir/faq/index-j.html' title='よくあるご質問（FAQ） | 鹿島建設株式会社' snippet='Q. 株主優待制度の有無について知りたい。 Q. 株主還元方針、配当金の推移について知りたい。' engine='startpage' score=0.5 extracted_at='2026-01-08T15:08:16.923505'
u

In [1]:
company_info = [
    {
        "company_name_en": "Lasertec corporation",
        "company_name": "レーザーテック株式会社",
        "company_code": "E01991",
        "stock_ticker": "69200",
    },
    {
        "company_name_en": "Shin-Etsu Chemical Co., Ltd.",
        "company_name": "信越化学工業株式会社",
        "company_code": "E00776",
        "stock_ticker": "40630",
    },
    {
        "company_name_en": "Mitsui O.S.K. Lines, Ltd.",
        "company_name": "株式会社　商船三井",
        "company_code": "E04236",
        "stock_ticker": "91040",
    },
    {
        "company_name_en": "DeNA Co., Ltd",
        "company_name": "株式会社ディー・エヌ・エー",
        "company_code": "E05460",
        "stock_ticker": "24320",
    },
    {
        "company_name_en": "CAPCOM CO., LTD.",
        "company_name": "株式会社カプコン",
        "company_code": "E02417",
        "stock_ticker": "96970",
    },
    {
        "company_name_en": "BRIDGESTONE CORPORATION",
        "company_name": "株式会社ブリヂストン",
        "company_code": "E01086",
        "stock_ticker": "51080",
    },
    {
        "company_name_en": "DAIICHI SANKYO COMPANY, LIMITED",
        "company_name": "第一三共株式会社",
        "company_code": "E00984",
        "stock_ticker": "45680",
    },
    {
        "company_name_en": "KAJIMA  CORPORATION",
        "company_name": "鹿島建設株式会社",
        "company_code": "E00058",
        "stock_ticker": "18120",
    },
    {
        "company_name_en": "JFE Holdings,Inc.",
        "company_name": "ＪＦＥホールディングス株式会社",
        "company_code": "E01264",
        "stock_ticker": "54110",
    },
    {
        "company_name_en": "Nintendo Co., Ltd.",
        "company_name": "任天堂株式会社",
        "company_code": "E02367",
        "stock_ticker": "79740",
    },
    {
        "company_name_en": "Mitsubishi Heavy Industries,Ltd.",
        "company_name": "三菱重工業株式会社",
        "company_code": "E02126",
        "stock_ticker": "70110",
    },
    {
        "company_name_en": "ITOCHU Corporation",
        "company_name": "伊藤忠商事株式会社",
        "company_code": "E02497",
        "stock_ticker": "80010",
    },
    {
        "company_name_en": "CASIO COMPUTER CO.,LTD.",
        "company_name": "カシオ計算機株式会社",
        "company_code": "E01935",
        "stock_ticker": "69520",
    },
    {
        "company_name_en": "Bank of Innovation, Inc.",
        "company_name": "株式会社バンク・オブ・イノベーション",
        "company_code": "E34116",
        "stock_ticker": "43930",
    },
    {
        "company_name_en": "Mitsubishi UFJ Financial Group, Inc.",
        "company_name": "株式会社三菱ＵＦＪフィナンシャル・グループ",
        "company_code": "E03606",
        "stock_ticker": "83060",
    },
    {
        "company_name_en": "ADVANTEST CORPORATION",
        "company_name": "株式会社アドバンテスト",
        "company_code": "E01950",
        "stock_ticker": "68570",
    },
    {
        "company_name_en": "TOYOTA MOTOR CORPORATION",
        "company_name": "トヨタ自動車株式会社",
        "company_code": "E02144",
        "stock_ticker": "72030",
    },
    {
        "company_name_en": "FAST RETAILING CO., LTD.",
        "company_name": "株式会社ファーストリテイリング",
        "company_code": "E03217",
        "stock_ticker": "99830",
    },
    {
        "company_name_en": "Mitsui Fudosan Co., Ltd.",
        "company_name": "三井不動産株式会社",
        "company_code": "E03855",
        "stock_ticker": "88010",
    },
    {
        "company_name_en": "SoftBank Group Corp.",
        "company_name": "ソフトバンクグループ株式会社",
        "company_code": "E02778",
        "stock_ticker": "99840",
    },
    {
        "company_name_en": "KOEI TECMO HOLDINGS CO., LTD.",
        "company_name": "株式会社コーエーテクモホールディングス",
        "company_code": "E22460",
        "stock_ticker": "36350",
    },
    {
        "company_name_en": "TOEI ANIMATION CO.,LTD.",
        "company_name": "東映アニメーション株式会社",
        "company_code": "E02458",
        "stock_ticker": "48160",
    },
    {
        "company_name_en": "ENEOS Holdings,Inc.",
        "company_name": "ＥＮＥＯＳホールディングス株式会社",
        "company_code": "E24050",
        "stock_ticker": "50200",
    },
    {
        "company_name_en": "Mitsubishi Corporation",
        "company_name": "三菱商事株式会社",
        "company_code": "E02529",
        "stock_ticker": "80580",
    },
    {
        "company_name_en": "LY Corporation",
        "company_name": "ＬＩＮＥヤフー株式会社",
        "company_code": "E05000",
        "stock_ticker": "46890",
    },
]

In [13]:
from pyserxng.models import SearchCategory, SafeSearchLevel, TimeRange, SearchConfig
from pyserxng import SearXNGClient
from pyserxng.models import InstanceInfo

client = SearXNGClient()
local_instance = InstanceInfo(url="http://localhost:8888")
results = client.search("AI research", instance=local_instance)

search_company = company_info[0]
search_text = f"{search_company['company_name']} -- 株主優待の有無・内容"
results = client.search(search_text, instance=local_instance)
addition_info = f"{search_company['company_code']} {search_company['stock_ticker']}"

2026-01-13 17:04:44,741 - pyserxng.client - INFO - Search completed: 21 results in 3.41s from http://localhost:8888/
2026-01-13 17:04:48,011 - pyserxng.client - INFO - Search completed: 20 results in 3.24s from http://localhost:8888/


In [14]:
results.results

[SearchResult(title='レーザーテック[6920]の株価・株主優待など。', url=HttpUrl('https://www.nikkei.com/nkd/company/?scode=6920'), content='上場企業の決算発表日程や株主総会の日程を事前に確認することができます。 ... 企業名や証券コード以外にも優待の種類やキーワードで検索できます。よく見られている優待情報も確認できます。', engine=None, category=None, score=None, thumbnail=None, publishedDate=None),
 SearchResult(title='レーザーテック（レーザーテク）【6920】株の基本情報｜株探（かぶたん）', url=HttpUrl('https://kabutan.jp/stock/?code=6920'), content='株主優待 | おすすめの優待情報 · 株探おすすめ証券会社比較 · クレジットカードおすすめランキング｜人気クレカを徹底比較 · カードローンおすすめランキング ·', engine=None, category=None, score=None, thumbnail=None, publishedDate=None),
 SearchResult(title='レーザーテックの株主優待 | 国内株式：株主優待 | マーケット情報 | 楽天証券', url=HttpUrl('https://www.rakuten-sec.co.jp/web/market/search/hp2/hp_6920.T.html'), content='ホーム > マーケット情報 > 国内株式銘柄検索 > 国内株式：レーザーテックの株主優待', engine=None, category=None, score=None, thumbnail=None, publishedDate=None),
 SearchResult(title='レーザーテック (6920) : 株の最少購入金額・売買手数料 [Lasertec] - みんかぶ', url=HttpUrl('https://minkabu.jp/stock/6920/fee'), content='配当・優待二刀流銘柄 ·